In [2]:
from scipy.sparse import *
from numpy import *

In [3]:
trackstags=load_npz("tracks_with_tags_bincount_ordered.npz")
trackstags

<100000x276615 sparse matrix of type '<class 'numpy.float64'>'
	with 487328 stored elements in Compressed Sparse Column format>

In [4]:
trackstags[0].nonzero()[1]

array([ 43212,  64978,  70618,  70625, 255208], dtype=int32)

In [24]:
prod=trackstags*trackstags.T # memory error

In [6]:
playtags=load_npz("playlists_with_tags.npz")
playtags

<10000x276615 sparse matrix of type '<class 'numpy.float64'>'
	with 559049 stored elements in Compressed Sparse Column format>

In [7]:
prodplay=playtags*playtags.T

In [8]:
type(prodplay)

scipy.sparse.csc.csc_matrix

In [9]:
prodplay

<10000x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 91965186 stored elements in Compressed Sparse Column format>

In [14]:
prodplay[0].data

array([    0.,   198.,    34., ...,  2283.,  3419.,  2181.])

In [11]:
prodplay.setdiag(0)

/home/oppy/.local/lib/python3.5/site-packages/scipy/sparse/compressed.py:774: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [15]:
max(prodplay[0].data)

18789.0

In [16]:
(prodplay[1]>max(prodplay[1].data)-1).nonzero()[1][0]

7805

In [60]:
argmax(prodplay[3])

248

In [27]:
zeroTag=0
for p in prodplay:
    index=argmax(p)
    if (index==0):
        zeroTag+=1

In [31]:
zeroTag# number of playlists without tags = 70

70

prendere canzoni della playlist più simile

prendere canzoni più popolari

controllare che non ci siano già

e se non ce ne sono 5 diverse?

e se la playlist non ha tag? usare i top general

In [32]:
ordered_indexes=load("indexsortedDESCENDENT.npy")
ordered_indexes

array([43529, 94738, 57501, ..., 17820, 79618, 43000])

In [38]:
where(ordered_indexes==57501)[0][0]

2

In [40]:
playlists_with_tracks=load("playlists_with_tracks.npy")
playlists_with_tracks

array([[10024884,  2879006,  1532328, ...,        0,        0,        0],
       [10624787,  2090319,  2152295, ...,        0,        0,        0],
       [ 4891851,  2621486,  2895498, ...,        0,        0,        0],
       ..., 
       [11369546,   416692,  1587449, ...,        0,        0,        0],
       [ 7939535,  2788577,  3090083, ...,        0,        0,        0],
       [  297021,  2789282,  2154608, ...,        0,        0,        0]])

In [43]:
pIDs=playlists_with_tracks[:,0]
pIDs

array([10024884, 10624787,  4891851, ..., 11369546,  7939535,   297021])

In [46]:
pTracks=playlists_with_tracks[:,1:]
pTracks

array([[2879006, 1532328, 3027673, ...,       0,       0,       0],
       [2090319, 2152295, 3230559, ...,       0,       0,       0],
       [2621486, 2895498, 1369381, ...,       0,       0,       0],
       ..., 
       [ 416692, 1587449, 1057389, ...,       0,       0,       0],
       [2788577, 3090083, 1609308, ...,       0,       0,       0],
       [2789282, 2154608,  973448, ...,       0,       0,       0]])

In [158]:
totalSugg=[]
for i,(pID,pT) in enumerate(zip(pIDs,pTracks)):
    index=argmax(prodplay[i])
    if (index!=0):
        mostSimilar=pIDs[index]
        toSuggest=list(pTracks[index])
        alreadyPresent=intersect1d(toSuggest,pT)
        for a in alreadyPresent:
            toSuggest.remove(a)
        bestTracksRankPositions=suggestionRank(toSuggest)
        suggestions=[]
        while (len(suggestions)<5):
            for b in bestTracksRankPositions:
                currentTrack=ordered_tracks[b]
                if (not isin(currentTrack,pT)):
                    suggestions.append(currentTrack)
                    bestTracksRankPositions.remove(b)
                    break
        totalSugg.append(suggestions)
    else:
        totalSugg.append(-1)
    

In [148]:
trim_zeros(toSuggest)

array([ 571279, 2378652, 3626793, 3871109,  793297, 1337992, 2090330,
       3455524, 1896558, 2439971,  686140, 1685106, 1327233, 2550759,
       1669771,  611456, 3664896,  723392,  852112, 3224828, 2093563,
        897771, 1713531,  416530, 2840855, 3861322, 3073582, 1560665,
       3173841,  245506, 3441295, 3343871, 2813027, 4733003, 2762367,
        798801, 2999747, 1105456, 2839393, 3033242, 1742602,  726679,
       2586490, 3855688, 2309023, 2584586, 3068991,  574364, 1627524,
       3526178, 1790459, 2912475, 1331563, 1137264,  195273,  883229,
       3460931,  462780, 3551510,  223071, 2905797, 1978398, 3876951,
       2534480, 3822926, 1074579, 1553844, 1931028, 3356950, 1494139,
       2392241, 2696397, 3361557,       0,  620358, 3557198, 1517584,
        675104,  310707,  968899, 3863544,  151738, 1839327,  159953,
       3715626, 2302904, 2741789, 3749974, 3779477, 2746443, 1887487,
        170826,  570624, 2454612, 3801835, 1563309,  414551, 4798096,
       1855833, 2889

In [ ]:

where()

In [73]:
ordered_indexes[0]

43529

In [74]:
ordered_indexes.shape

(100000,)

In [95]:
ordered_tracks=array(zeros(100000))

In [72]:
tracks_with_tags=load("tracks_with_tags.npy")
tracks_with_tags.shape

(100000, 6)

In [83]:
trackIDs=tracks_with_tags[:,0]
trackIDs

array([2972914, 2750239, 1550729, ..., 2584455, 2299706, 2739985], dtype=int32)

In [94]:
ordered_indexes[99999]

43000

In [115]:
ordered_tracks[1]

1363985.0

In [124]:
where(ordered_tracks==toSuggest[4])[0][0]

3643

In [98]:
for i,j in enumerate(ordered_indexes):
    ordered_tracks[i]=trackIDs[j]

In [104]:
save("ordered_tracks.npy",ordered_tracks)

In [140]:
def suggestionRank(toSuggest):
    tempList=[]
    for t in trim_zeros(toSuggest):
        tempList.append(where(ordered_tracks==t)[0][0])
    return sorted(tempList)

In [143]:
suggestionRank(toSuggest)

[0,
 12,
 26,
 58,
 127,
 139,
 190,
 243,
 260,
 267,
 325,
 438,
 443,
 456,
 466,
 516,
 518,
 543,
 548,
 587,
 624,
 633,
 641,
 648,
 727,
 735,
 769,
 800,
 804,
 830,
 873,
 891,
 1026,
 1056,
 1079,
 1084,
 1178,
 1279,
 1328,
 1365,
 1387,
 1423,
 1465,
 1466,
 1653,
 1755,
 1799,
 1803,
 1909,
 1940,
 2252,
 2371,
 2594,
 2797,
 2877,
 2882,
 2959,
 2991,
 3238,
 3282,
 3333,
 3599,
 3643,
 3671,
 3790,
 3967,
 4030,
 4042,
 4127,
 4264,
 4287,
 4644,
 4673,
 5029,
 5060,
 5239,
 5290,
 5476,
 6117,
 6261,
 6464,
 6994,
 7009,
 7139,
 7325,
 7564,
 8018,
 8145,
 8420,
 8453,
 8592,
 8621,
 8690,
 9619,
 9805,
 10071,
 10373,
 10781,
 11043,
 11154,
 11157,
 12191,
 13325,
 13512,
 13527,
 14585,
 14597,
 14771,
 15423,
 16452,
 16799,
 19100,
 19572,
 19600,
 21878,
 22300,
 22321,
 22674,
 25104,
 25532,
 25854,
 25885,
 27620,
 28469,
 29242,
 31005,
 31131,
 32470,
 33670,
 33671,
 38623,
 42255,
 47374,
 47664,
 57647,
 57658,
 57666,
 76630,
 82132,
 91737,
 91742]

In [170]:
totalSugg[9]

[3705881.0, 3794036.0, 1705891.0, 1218315.0, 3542444.0]

In [159]:
type(toSuggest)

list

In [171]:
len(totalSugg)

10000

In [178]:
pIDs.shape

(10000,)

In [182]:
pIDs[0]

10024884

In [206]:
arrayTotal=array(totalSugg)

In [207]:
save("suggestionPLsimilarity.npy",arrayTotal)

In [197]:
where(arrayTotal==-1)

(array([  35,   38,   96,  201,  306,  360,  583,  841,  861, 1188, 1202,
        1251, 1541, 1678, 1934, 1989, 2063, 2105, 2151, 2157, 2168, 2311,
        2445, 2604, 3269, 3439, 3560, 3635, 4430, 4788, 4917, 5081, 5166,
        5167, 5495, 6283, 6583, 6675, 6894, 7057, 7075, 7126, 7174, 7372,
        7415, 7631, 7648, 7660, 7693, 7742, 7757, 7769, 7976, 8197, 8298,
        8411, 8546, 8552, 8615, 8791, 9153, 9156, 9346, 9369, 9484, 9730,
        9759, 9817, 9856, 9898]),)

In [200]:
pIDs[96]

7947658

In [186]:
totalSugg[0][0]

1563309.0

In [203]:
def cleanStr(id,ts):
    myStr=""
    myStr+=str(id)
    myStr+=","
    if (ts!=-1):
        for t in ts:
            myStr+=" "
            myStr+=str(int(t))
    else:
        myStr+=" 1563309 1363985 3705881 1595978 3166665"
    myStr+="\n"
    return myStr

In [205]:
with open("playlistSimilar.csv","a") as myFile:
    for id,ts in zip(pIDs,totalSugg):
        #print(id,ts)
        myFile.write(cleanStr(id,ts))
    